# Model Evaluation

Instructor: Sabyasachi Dasgupta

## Introduction
In this notebook, we will go through the process of creating a statistical model and evaluating its goodness of fit. We will do this for both a regression and classification model (linear and logistic regression).

## Model Identification
- EDA and data cleaning is "done" at this point.
- What is our dependent variable?
- What are our (potential) independent variables?
- Does this fit into a regression or classification framework?
- What statistical models can we use (linear regression or logistic regression for now)
    - Check model assumptions

## Regression Example

In this example, we will use a used car dataset. The dependent variable is 'selling_price' with all other variables being potential independent variables.

### Load the dataset

The dataset is stored locally as a csv. Take some time to familiarize yourself with the dataset. Assume the data cleaning has already been performed.

In [1]:
import pandas as pd

car_df = pd.read_csv("data/UsedCars2.csv")
car_df.head()

,selling_price,year,km_driven,owners,kmpl,engine_cc,power_bhp,seats
0,130000,2007,120000,1,16.10,1298,88.2,5
1,778000,2016,70000,2,24.52,1248,88.5,7
2,500000,2012,53000,2,23.00,1396,90.0,5
3,600000,2012,72000,1,21.50,1248,88.8,5
4,1149000,2019,5000,1,17.00,1591,121.3,5


In [3]:
car_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1595 entries, 0 to 1594
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   selling_price  1595 non-null   int64  
 1   year           1595 non-null   int64  
 2   km_driven      1595 non-null   int64  
 3   owners         1595 non-null   int64  
 4   kmpl           1595 non-null   float64
 5   engine_cc      1595 non-null   int64  
 6   power_bhp      1595 non-null   float64
 7   seats          1595 non-null   int64  
dtypes: float64(2), int64(6)
memory usage: 99.8 KB


In [4]:
car_df.describe()

,selling_price,year,km_driven,owners,kmpl,engine_cc,power_bhp,seats
count,1.595000e+03,1595.000000,1595.000000,1595.000000,1595.000000,1595.000000,1595.000000,1595.000000
mean,6.963078e+05,2014.269592,67922.847649,1.431348,19.409555,1465.480878,93.251586,5.423824
std,8.844260e+05,3.698153,50196.275311,0.705434,3.904365,499.842407,35.342834,0.956879
min,4.000000e+04,1997.000000,1000.000000,0.000000,10.000000,624.000000,34.200000,4.000000
25%,2.800000e+05,2012.000000,32000.000000,1.000000,16.780000,1197.000000,70.000000,5.000000
50%,4.849990e+05,2015.000000,60000.000000,1.000000,19.300000,1248.000000,84.000000,5.000000
75%,7.000000e+05,2017.000000,90000.000000,2.000000,22.320000,1588.500000,103.600000,5.000000
max,6.223000e+06,2020.000000,500000.000000,4.000000,28.400000,3198.000000,261.400000,10.000000


### Building Models

We are now ready to start building a linear regression model! We are probably interested in how selling price depends on the other variables. Selling price is our dependent variable, but which independent variables should we use? All of them? A select few? Which will be the best model? There are a few strategies we could implement.

#### Forward Selection
1. Build all models that contain only one independent variable.
    - $selling\_price = b_0+b_1(year)$
    - $selling\_price = b_0+b_1(km\_driven)$
    - $selling\_price = b_0+b_1(fuel)$
    - etc...
2. Identify the best model (maybe highest adj. R<sup>2</sup> value) and then add one more variable at a time.
3. Find the new best model.
4. Repeat until you hit some stopping criteria (maybe adj. R<sup>2</sup> stops increasing or you get p-values > 0.05).

![](images/forward_selection.png)



In [19]:
import statsmodels.api as sm

y = car_df['selling_price']
indep = car_df.drop('selling_price', axis=1)

# X=[]
# for column in indep.columns:
#     X.append(sm.add_constant(indep[column]))

#Create a model for each indep. variable
#list of X's (with constants)
X = [sm.add_constant(indep[column]) for column in indep.columns] 
#Print all the X columns in a formatted manner as pandas dataframe
df_X = pd.DataFrame(indep, columns=indep.columns)
df_X.head()
 

,year,km_driven,owners,kmpl,engine_cc,power_bhp,seats
0,2007,120000,1,16.10,1298,88.2,5
1,2016,70000,2,24.52,1248,88.5,7
2,2012,53000,2,23.00,1396,90.0,5
3,2012,72000,1,21.50,1248,88.8,5
4,2019,5000,1,17.00,1591,121.3,5


In [21]:
Models = [sm.OLS(y,x) for x in X] #list of models
Results = [model.fit() for model in Models] #list of results
Adj_Rsquared = [results.rsquared_adj for results in Results] #list of rsquared
Pval = [results.pvalues for results in Results] #list of p-values
Params = [results.params for results in Results] #list of params

In [33]:
#Built a Model for each indep. variable and then summarize the results
for i in range(len(indep.columns)):
    print("Model for", indep.columns[i])
    print(Results[i].summary())
    print(" ")  #print a blank line

#Create a dataframe with the indep. variables, their adj. R-squared, p-values and params
df = pd.DataFrame({'Indep. Variables': indep.columns, 'Adj. R-squared': Adj_Rsquared, 'P-values': Pval, 'Params': Params})
df

#Create a dataframe with the indep. variables, their adj. R-squared, p-values and params
df = pd.DataFrame({'Indep. Variables': indep.columns, 'Adj. R-squared': Adj_Rsquared, 'P-values': Pval, 'Params': Params})

#Sort the dataframe by the adj. R-squared
df.sort_values(by='Adj. R-squared', ascending=False)

#Sort the dataframe by the adj. R-squared
df.sort_values(by='Adj. R-squared', ascending=False)

    

Model for year
                            OLS Regression Results                            
Dep. Variable:          selling_price   R-squared:                       0.174
Model:                            OLS   Adj. R-squared:                  0.174
Method:                 Least Squares   F-statistic:                     336.0
Date:                Fri, 24 Mar 2023   Prob (F-statistic):           2.98e-68
Time:                        12:13:30   Log-Likelihood:                -23950.
No. Observations:                1595   AIC:                         4.790e+04
Df Residuals:                    1593   BIC:                         4.791e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -2.004e+08    1.1e+07    -

,Indep. Variables,Adj. R-squared,P-values,Params
5,power_bhp,0.579454,const 9.412783e-130 power_bhp 3.7110...,const -1.080435e+06 power_bhp 1.90532...
4,engine_cc,0.198786,const 9.805585e-14 engine_cc 5.86811...,const -461269.506189 engine_cc 789...
0,year,0.173670,const 7.813010e-68 year 2.981846e-68 dt...,const -2.003539e+08 year 9.981294e+04 dt...
1,km_driven,0.070022,const 1.913409e-142 km_driven 3.501...,const 1.014306e+06 km_driven -4.68176...
2,owners,0.057031,const 1.518654e-102 owners 2.433141e-2...,const 1.127080e+06 owners -3.009556e+05 ...
3,kmpl,0.013418,const 8.935450e-27 kmpl 2.088159e-06 dt...,const 1.217218e+06 kmpl -2.683781e+04 dt...
6,seats,-0.000246,const 0.000003 seats 0.435925 dtype: flo...,const 598444.849988 seats 18043.163365 ...


In [34]:
for i in range(len(Adj_Rsquared)):
     print(f'adj_R2: {Adj_Rsquared[i]:.3f}, P-values: {*Pval[i],}, column: {indep.columns[i]}')

adj_R2: 0.174, P-values: (7.813010353972577e-68, 2.9818464289461137e-68), column: year
adj_R2: 0.070, P-values: (1.913408757155729e-142, 3.501311660968875e-27), column: km_driven
adj_R2: 0.057, P-values: (1.5186541212434708e-102, 2.433141313618363e-22), column: owners
adj_R2: 0.013, P-values: (8.935450361427155e-27, 2.0881588197525646e-06), column: kmpl
adj_R2: 0.199, P-values: (9.805584670007918e-14, 5.868115795936577e-79), column: engine_cc
adj_R2: 0.579, P-values: (9.412782556143103e-130, 3.7110127230227896e-302), column: power_bhp
adj_R2: -0.000, P-values: (2.922458353965922e-06, 0.4359248539986539), column: seats


Note: The adjusted R-squared is a modified version of R-squared that adjusts for predictors that are not significant in a regression model. It is calculated as 1 - [(1 - R^2) * (n - 1) / (n - k - 1)] where n is the number of observations and k is the number of independent variables.  


https://statisticsbyjim.com/regression/interpret-adjusted-r-squared-predicted-r-squared-regression/
https://corporatefinanceinstitute.com/resources/data-science/adjusted-r-squared/


From this output, we can see that the model with 'power_bhp' had the highest adj. R-squared value. Now let's try all models that consist of 'power_bhp' and another variable.

In [35]:
remaining_var = car_df.drop(['selling_price', 'power_bhp'], axis=1)
remaining_var.head()

,year,km_driven,owners,kmpl,engine_cc,seats
0,2007,120000,1,16.10,1298,5
1,2016,70000,2,24.52,1248,7
2,2012,53000,2,23.00,1396,5
3,2012,72000,1,21.50,1248,5
4,2019,5000,1,17.00,1591,5


In [36]:
included_df = car_df[['power_bhp']]
included_df

,power_bhp
0,88.20
1,88.50
2,90.00
3,88.80
4,121.30
...,...
1590,67.04
1591,67.06
1592,102.00
1593,37.00


In [37]:
X = [sm.add_constant(pd.merge(included_df,remaining_var[column], right_index = True, left_index = True)) for column in remaining_var.columns] 
X[0]

,const,power_bhp,year
0,1.0,88.20,2007
1,1.0,88.50,2016
2,1.0,90.00,2012
3,1.0,88.80,2012
4,1.0,121.30,2019
...,...,...,...
1590,1.0,67.04,2017
1591,1.0,67.06,2014
1592,1.0,102.00,2010
1593,1.0,37.00,1997


In [38]:
Models = [sm.OLS(y,x) for x in X] #list of models
Results = [model.fit() for model in Models] #list of results
Adj_Rsquared = [results.rsquared_adj for results in Results] #list of rsquared
Pval = [results.pvalues for results in Results] #list of list of p-values

for i in range(len(Adj_Rsquared)):
     print(f'adj_R2: {Adj_Rsquared[i]:.3f}, P-values: {*Pval[i],}, column: {remaining_var.columns[i]}')

adj_R2: 0.639, P-values: (1.0016051895194447e-55, 1.2162697843801444e-288, 7.61190755401513e-55), column: year
adj_R2: 0.630, P-values: (2.978237375005332e-67, 2.737e-321, 1.357509435339205e-46), column: km_driven
adj_R2: 0.605, P-values: (2.0502564956273112e-46, 5.118641374948136e-303, 2.1911838330018783e-23), column: owners
adj_R2: 0.618, P-values: (2.9293658745854524e-99, 0.0, 3.62055664918882e-35), column: kmpl
adj_R2: 0.596, P-values: (1.72841629958627e-80, 2.17291507649577e-239, 5.051132565766585e-16), column: engine_cc
adj_R2: 0.595, P-values: (3.0827264856713875e-09, 1.012762537e-314, 1.3612693239730793e-14), column: seats


It looks like 'power_bhp' with 'year' is now the "best" model. Keep repeating this process until the adjusted R<sup>2</sup> stops increasing by a significant amount, or we get insignificant indep. variables in our model.

In [39]:
remaining_var = car_df.drop(['selling_price', 'power_bhp', 'year'], axis=1)
included_df = car_df[['power_bhp', 'year']]

In [40]:
X = [sm.add_constant(pd.merge(included_df,remaining_var[column], right_index = True, left_index = True)) for column in remaining_var.columns] 
X[3]

,const,power_bhp,year,engine_cc
0,1.0,88.20,2007,1298
1,1.0,88.50,2016,1248
2,1.0,90.00,2012,1396
3,1.0,88.80,2012,1248
4,1.0,121.30,2019,1591
...,...,...,...,...
1590,1.0,67.04,2017,998
1591,1.0,67.06,2014,1364
1592,1.0,102.00,2010,2494
1593,1.0,37.00,1997,796


In [41]:
Models = [sm.OLS(y,x) for x in X] #list of models
Results = [model.fit() for model in Models] #list of results
Adj_Rsquared = [results.rsquared_adj for results in Results] #list of rsquared
Pval = [results.pvalues for results in Results] #list of list of p-values

for i in range(len(Adj_Rsquared)):
     print(f'adj_R2: {Adj_Rsquared[i]:.3f}, P-values: {*Pval[i],}, column: {remaining_var.columns[i]}')

adj_R2: 0.653, P-values: (1.381643915639696e-24, 1.1786444978228554e-300, 3.424871367226014e-24, 6.940265994715546e-16), column: km_driven
adj_R2: 0.640, P-values: (3.153355279090331e-35, 9.663792642463707e-290, 8.600152533654988e-35, 0.006191716039320925), column: owners
adj_R2: 0.647, P-values: (3.2587569499234126e-31, 1.0195263014648635e-265, 5.950215985025602e-30, 3.909174785917899e-10), column: kmpl
adj_R2: 0.644, P-values: (2.3233700604739115e-46, 2.5071744097662626e-198, 1.085637913861393e-45, 1.0046581761525204e-06), column: engine_cc
adj_R2: 0.650, P-values: (5.766392969576132e-53, 1.2532079582573496e-298, 1.4546564360142364e-52, 2.635421899881347e-12), column: seats


Next include power_bhp, year, km_driven and one other variable.

In [42]:
remaining_var = car_df.drop(['selling_price', 'power_bhp', 'year', 'km_driven'], axis=1)
included_df = car_df[['power_bhp', 'year', 'km_driven']]

In [43]:
X = [sm.add_constant(pd.merge(included_df,remaining_var[column], right_index = True, left_index = True)) for column in remaining_var.columns] 
X[0]

,const,power_bhp,year,km_driven,owners
0,1.0,88.20,2007,120000,1
1,1.0,88.50,2016,70000,2
2,1.0,90.00,2012,53000,2
3,1.0,88.80,2012,72000,1
4,1.0,121.30,2019,5000,1
...,...,...,...,...,...
1590,1.0,67.04,2017,12000,1
1591,1.0,67.06,2014,50000,1
1592,1.0,102.00,2010,129000,1
1593,1.0,37.00,1997,120000,1


In [44]:
Models = [sm.OLS(y,x) for x in X] #list of models
Results = [model.fit() for model in Models] #list of results
Adj_Rsquared = [results.rsquared_adj for results in Results] #list of rsquared
Pval = [results.pvalues for results in Results] #list of list of p-values

for i in range(len(Adj_Rsquared)):
     print(f'adj_R2: {Adj_Rsquared[i]:.3f}, P-values: {*Pval[i],}, column: {remaining_var.columns[i]}')

adj_R2: 0.654, P-values: (1.402538668543844e-17, 4.446218328399372e-301, 2.4623940605873746e-17, 4.820929991330839e-15, 0.054019726034460674), column: owners
adj_R2: 0.661, P-values: (4.5204160861386957e-13, 1.1364963747485822e-275, 1.6934225013472338e-12, 2.22346407136054e-15, 1.2561337248200949e-09), column: kmpl
adj_R2: 0.654, P-values: (4.697111808408104e-24, 2.525630867161833e-184, 1.1055318498154255e-23, 8.273175074222188e-12, 0.0190720293894852), column: engine_cc
adj_R2: 0.658, P-values: (2.975609559074873e-26, 1.4477809213489476e-304, 5.446723779109058e-26, 1.4285901026148458e-10, 5.954371587038121e-07), column: seats


Hopefully you get the idea. We eventually settle on a "best model". This whole process of forward selection can be quite time consuming! Usually a different selection method is preferred:

#### Backward Selection
1. Build a model that contains all variables
    - $selling\_price = b_0+b_1(year) + b_2(km\_driven) + b_3(fuel) + \ldots$
2. Remove one variable at a time usually based on p-value.
3. Repeat until you hit some stopping criteria (maybe adj. R<sup>2</sup> stops increasing).

![](images/backward_selection.png)



In [45]:
#run full model
y = car_df['selling_price']
X = car_df.drop('selling_price', axis=1)
X = sm.add_constant(X) #adds a column of 1's so the model will contain an intercept

model = sm.OLS(y, X)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:          selling_price   R-squared:                       0.665
Model:                            OLS   Adj. R-squared:                  0.663
Method:                 Least Squares   F-statistic:                     449.6
Date:                Fri, 24 Mar 2023   Prob (F-statistic):               0.00
Time:                        12:21:39   Log-Likelihood:                -23231.
No. Observations:                1595   AIC:                         4.648e+04
Df Residuals:                    1587   BIC:                         4.652e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       -6.48e+07   1.02e+07     -6.354      0.0

Notice the Adj. R-Squared and the p-values for each of the coefficients. The coefficient for 'owners' seems to be the only variable that has a p-value > 0.05. Let's try removing it and running another model.

In [46]:
y = car_df['selling_price']
X = car_df.drop(['selling_price', 'owners'], axis=1)
X = sm.add_constant(X) #adds a column of 1's so the model will contain an intercept

model = sm.OLS(y, X)
results = model.fit() #fit the model (this is where OLS is actually being run)
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:          selling_price   R-squared:                       0.664
Model:                            OLS   Adj. R-squared:                  0.663
Method:                 Least Squares   F-statistic:                     523.0
Date:                Fri, 24 Mar 2023   Prob (F-statistic):               0.00
Time:                        12:22:06   Log-Likelihood:                -23233.
No. Observations:                1595   AIC:                         4.648e+04
Df Residuals:                    1588   BIC:                         4.652e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -7.258e+07   9.38e+06     -7.735      0.0

At this point, we'd probably just stick with this model. All coefficients have significant p-values.

#### Stepwise Selection
- A combination of forward and backward selection.

#### Forward vs Backward Selection
- Forward selection is better when there are more variables than datapoints. Otherwise backward selection is typically used.
- Backward selection can consider the effects of all variables simultaneously.
- Different selection methods can give different results. An inexact science.
    - Notice in the backward selection model, engine_cc would have been the next to be eliminated if sig. level=0.01 instead of 0.05
- Over this process you'll find out which variables tend to get eliminated and which tend to be kept (hopefully).

In [58]:
#Use RFE to sleect the best 3 features in car_df 
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

#Create a linear regression object
lm = LinearRegression()
# RFE with 3 features selected
rfe = RFE(lm, n_features_to_select=3 )
rfe = rfe.fit(X, y)

#Print the ranking of each feature
print(rfe.ranking_)
print(rfe.support_)
print(X.columns)

#Create a dataframe with the features and their ranking
df = pd.DataFrame({'Features': X.columns, 'Ranking': rfe.ranking_})
df

#Sort the dataframe by ranking
df.sort_values(by='Ranking')


 

[2 1 1 1]
[False  True  True  True]
Index(['const', 'power_bhp', 'year', 'km_driven'], dtype='object')


,Features,Ranking
1,power_bhp,1
2,year,1
3,km_driven,1
0,const,2


In [59]:

# No show summary of the fit model on the sleected features
X = car_df[['power_bhp', 'year', 'km_driven']]
X = sm.add_constant(X) #adds a column of 1's so the model will contain an intercept

model = sm.OLS(y, X)
results = model.fit() #fit the model (this is where OLS is actually being run)

print(results.summary())




                            OLS Regression Results                            
Dep. Variable:          selling_price   R-squared:                       0.654
Model:                            OLS   Adj. R-squared:                  0.653
Method:                 Least Squares   F-statistic:                     1001.
Date:                Fri, 24 Mar 2023   Prob (F-statistic):               0.00
Time:                        12:28:19   Log-Likelihood:                -23257.
No. Observations:                1595   AIC:                         4.652e+04
Df Residuals:                    1591   BIC:                         4.654e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       -8.77e+07   8.43e+06    -10.408      0.0

## Classification Example

The exact same process can be used when building a logistic regression model. Perhaps we only have information on if the selling price of a car is greater than 500,000 or not.

In [60]:
car_df.head()

,selling_price,year,km_driven,owners,kmpl,engine_cc,power_bhp,seats
0,130000,2007,120000,1,16.10,1298,88.2,5
1,778000,2016,70000,2,24.52,1248,88.5,7
2,500000,2012,53000,2,23.00,1396,90.0,5
3,600000,2012,72000,1,21.50,1248,88.8,5
4,1149000,2019,5000,1,17.00,1591,121.3,5


In [61]:
import numpy as np
car_df['expensive'] = np.where(car_df['selling_price']>500000, 1, 0)
car_exp_df = car_df.drop(['selling_price'], axis=1)
car_exp_df

,year,km_driven,owners,kmpl,engine_cc,power_bhp,seats,expensive
0,2007,120000,1,16.10,1298,88.20,5,0
1,2016,70000,2,24.52,1248,88.50,7,1
2,2012,53000,2,23.00,1396,90.00,5,0
3,2012,72000,1,21.50,1248,88.80,5,1
4,2019,5000,1,17.00,1591,121.30,5,1
...,...,...,...,...,...,...,...,...
1590,2017,12000,1,23.10,998,67.04,5,0
1591,2014,50000,1,23.59,1364,67.06,5,0
1592,2010,129000,1,12.80,2494,102.00,8,0
1593,1997,120000,1,16.10,796,37.00,4,0


So 'expensive' is our binary dependent variable and we will consider the rest as potential independent variables. Let's use backwards selection for convenience :)

In [62]:
y = car_exp_df['expensive']
X  = car_exp_df.drop(['expensive'], axis=1)
X = sm.add_constant(X) #adds a column of 1's so the model will contain an intercept

model = sm.Logit(y,X)
results = model.fit()
print(results.summary())

Optimization terminated successfully.
         Current function value: 0.304869
         Iterations 9
                           Logit Regression Results                           
Dep. Variable:              expensive   No. Observations:                 1595
Model:                          Logit   Df Residuals:                     1587
Method:                           MLE   Df Model:                            7
Date:                Fri, 24 Mar 2023   Pseudo R-squ.:                  0.5592
Time:                        12:28:29   Log-Likelihood:                -486.27
converged:                       True   LL-Null:                       -1103.1
Covariance Type:            nonrobust   LLR p-value:                3.753e-262
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1405.5027    100.711    -13.956      0.000   -1602.893   -1208.113
year           0.6897      0.

The p-value for the km_driven coefficient is >0.05. Let's try removing that from the model.

In [63]:
y = car_exp_df['expensive']
X  = car_exp_df.drop(['expensive', 'km_driven'], axis=1)
X = sm.add_constant(X) #adds a column of 1's so the model will contain an intercept

model = sm.Logit(y,X)
results = model.fit()
print(results.summary())

Optimization terminated successfully.
         Current function value: 0.304897
         Iterations 9
                           Logit Regression Results                           
Dep. Variable:              expensive   No. Observations:                 1595
Model:                          Logit   Df Residuals:                     1588
Method:                           MLE   Df Model:                            6
Date:                Fri, 24 Mar 2023   Pseudo R-squ.:                  0.5591
Time:                        12:28:33   Log-Likelihood:                -486.31
converged:                       True   LL-Null:                       -1103.1
Covariance Type:            nonrobust   LLR p-value:                2.625e-263
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1394.8604     94.087    -14.825      0.000   -1579.267   -1210.454
year           0.6844      0.

Looks like this can be our final model!